# Importing and reading data from the GCP bucket

In [69]:
from pyspark.sql import SparkSession

In [70]:
spark = SparkSession.builder.appName('logistic_regression').getOrCreate()

In [71]:
training = spark.read.csv("gs://marketingbucket5502/customer_churn.csv")

In [72]:
training.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|                _c0| _c1|           _c2|            _c3|  _c4|      _c5|                _c6|                 _c7|                 _c8|  _c9|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|     

In [73]:
training = spark.read.option("header",True).csv("gs://marketingbucket5502/customer_churn.csv")

In [74]:
training.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

# EDA

In [75]:
training.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Total_Purchase: string (nullable = true)
 |-- Account_Manager: string (nullable = true)
 |-- Years: string (nullable = true)
 |-- Num_Sites: string (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: string (nullable = true)



In [76]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

training = training.withColumn("Age",training['Age'].cast(IntegerType()))
training = training.withColumn("Total_Purchase",training['Total_Purchase'].cast(DoubleType()))
training = training.withColumn("Account_Manager",training['Account_Manager'].cast(IntegerType()))
training = training.withColumn("Years",training['Years'].cast(DoubleType()))
training = training.withColumn("Num_Sites",training['Num_Sites'].cast(IntegerType()))
training = training.withColumn("Churn",training['Churn'].cast(IntegerType()))

In [77]:
training.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: integer (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [78]:
training.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [49]:
training.head()

Row(Names='Cameron Williams', Age=42, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [50]:
for i in training.head():
    print(i)

Cameron Williams
42
11066.8
0
7.22
8
2013-08-30 07:00:40
10265 Elizabeth Mission Barkerburgh, AK 89518
Harvey LLC
1


In [51]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [52]:
training.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [53]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [54]:
output = assembler.transform(training)

In [55]:
training_final = output.select('features','churn')

In [56]:
training_final.show(10)

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
+--------------------+-----+
only showing top 10 rows



# Train test split

In [57]:
train_churn,test_churn = training_final.randomSplit([0.7,0.3])

# Creating the logistic regression model and fitting it to the train dataset

In [58]:
from pyspark.ml.classification import LogisticRegression

In [113]:
logistic_regression_model_churn = LogisticRegression(labelCol='churn')

In [114]:
fitted_churn_model = logistic_regression_model_churn.fit(train_churn)

In [115]:
training_sum = fitted_churn_model.summary

In [116]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                612|                612|
|   mean|0.16830065359477125|0.13398692810457516|
| stddev|0.37443909536405473|0.34091691107127253|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



# Evaluaing the created model by predicting the churn for the test data and comparing it with original values

In [63]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [117]:
predictions_probabilities = fitted_churn_model.evaluate(test_churn)

In [118]:
predictions_probabilities.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,8670.98,0.0...|    0|[7.56276233235990...|[0.99948083127739...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.29041195312413...|[0.78421690835572...|       0.0|
|[29.0,5900.78,1.0...|    0|[3.61836191690838...|[0.97387427916765...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.38956562664405...|[0.91602816189436...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.36576123697989...|[0.98745441145884...|       0.0|
|[29.0,13255.05,1....|    0|[3.96289661986611...|[0.98134658800735...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.51142650809512...|[0.99597591896125...|       0.0|
|[30.0,8677.28,1.0...|    0|[3.65080982573064...|[0.97468728446140...|       0.0|
|[30.0,11575.37,1....|    1|[3.72484777657801...|[0.97645115092236...|       0.0|
|[30.0,13473.35,

In [119]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [120]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [121]:
auc

0.7535093140284277

# Using the created model for predicting the churn on the brand new label

In [122]:
logistic_regression_model_churn1 = logistic_regression_model_churn.fit(training_final)

In [123]:
customer_data_new = spark.read.csv('gs://marketingbucket5502/new_customers.csv',header=True,inferSchema=True)

In [124]:
customer_data_new.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [125]:
customer_data_new.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [126]:
customer_data_new1 = assembler.transform(customer_data_new)

In [127]:
customer_data_new1.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [128]:
customer_data_new1.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [129]:
customer_data_new_results = logistic_regression_model_churn1.transform(customer_data_new1)

In [130]:
customer_data_new_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

